In [42]:
import os
os.environ["CSDHOME"] = "/Users/siddhant/CCDC2025/ccdc-data/csd"

In [43]:
from ccdc import io 
csd_reader = io.EntryReader("CSD")
print(f"Number of entries: {len(csd_reader)}")

Number of entries: 1371757


In [45]:
# generate SMILES string based on REFcode
io.MoleculeReader("csd").molecule("ABEHUK").components[1].to_string('smiles')

'c1cc:[B-](:cc1)/C=C/c1ccc(cc1)/C=C/[B-]1:ccccc:1'

In [67]:
# search CSD for structures based on SMARTS match
import ccdc.search
queries = ccdc.search.SubstructureSearch()
queries.add_substructure(ccdc.search.SMARTSSubstructure("[NX3;H1]([CX4,c])[c]")) # secondaary aromatic amine
hits = search.search()
hits[0].identifier

'ABACIR'

In [79]:
MolecularDescriptors.

TypeError: MolecularDescriptors.atom_angle() missing 2 required positional arguments: 'b' and 'c'

In [93]:
csd = io.MoleculeReader('csd')
csd[0].smiles

'CC(=O)NN1C=NN=C1N(N=Cc1c(Cl)cccc1Cl)C(C)=O'